In [2]:
from utils import process_matchms as _myfun
import plotly.express as px
import pickle
import kmedoids
import numpy as np
from sklearn import manifold
import itertools
from plotly.colors import n_colors
import pandas as pd


In [ ]:

# Load similarity matrices.
file = open("data/sim_matrix_ms2deepscore", 'rb')
sm = pickle.load(file) # sm = _myfun.extract_similarity_scores(sm) # <- if cosine based
file.close()
dist = 1.-sm
dist = np.round(dist, 5)

# Process cosine

modified cosine sm matrix is actual a matrix of tuples with frag overlap and sim score.

In [ ]:
file = open("data/sim_matrix_cosine_modified", 'rb')
sm = pickle.load(file) 
sm = _myfun.extract_similarity_scores(sm)
file.close()

with open('data/sim_matrix_cosine_modified_extracted.pickle', 'wb') as handle:
    pickle.dump(sm, handle)

# Construct Class Table

In [ ]:
# Load Pandas Classification Table & Extract classes
file = open("data/pandas-classification-table-0012.pickle", 'rb')
class_table = pickle.load(file)
file.close()
#print(class_table.shape)
#print(class_table.iloc[0])
class_table = class_table.replace(" ","_",regex=True )
print(class_table.iloc[0])
for k in [10, 20, 25, 30, 35, 40, 45, 50, 100]:
    # kmedoid clustering
    cluster = kmedoids.KMedoids(n_clusters=k, metric='precomputed', random_state=0, method = "fasterpam")  
    cluster_km = cluster.fit_predict(dist)
    cluster_km = ["KM_" + str(elem) for elem in cluster_km]
    class_table[f"kmedoid_{k}"] = cluster_km

class_table.head()
print(list(class_table.columns))
# class_table = class_table.drop("npc_isglycoside", axis = 1)
# class_table = class_table.iloc[:, 3:]

print(list(class_table.columns))

class_table.to_csv('data/classification_table.csv', encoding='utf-8', index = False)

# Generate TSNE Embedding

In [ ]:
# Compute T-sne embedding
model = manifold.TSNE(metric="precomputed", random_state = 123, perplexity= 15)
z = model.fit_transform(dist)
x_coords = z[:,0] * 100 # scale the coordinate space for better visualization
y_coords = z[:,1] * 100 # scale the coordinate space for better visualization
id = np.arange(0, len(x_coords), 1)
tsne_df = pd.DataFrame({"x": x_coords, "y":y_coords, "id":id})

with open('data/tsne_df.pickle', 'wb') as handle:
    pickle.dump(tsne_df, handle)

# Generate precursor mz values vector

In [11]:
file = open("data/cleaned_demo_data.pickle", 'rb')
spectra = pickle.load(file) 
file.close()
spectrum = spectra[0]

mz = [round(spectrum.get("precursor_mz"), 2) for spectrum in spectra]
mz = np.array(mz)

with open('data/extracted_precursor_mz_values.pickle', 'wb') as handle:
    pickle.dump(mz, handle)